In [19]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
import math

In [20]:
model_path = r'E:\PosePerfect\Model\mobilenetV3_checkpoints\model_epoch_07_val_loss_0.36.keras'
cnn_lstm_model = load_model(model_path)
print("Model loaded successfully!")

Model loaded successfully!


In [21]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [22]:
def preprocess_video(frames, target_shape=(15, 224, 224, 3)):
    # Resize frames and stack into a batch
    processed_frames = [cv2.resize(frame, (224, 224)) for frame in frames]
    processed_frames = np.array(processed_frames, dtype='float32')
    processed_frames = preprocess_input(processed_frames)
    return np.expand_dims(processed_frames, axis=0)

In [23]:
# Function to predict activity from video frames
def predict_activity(frames):
    input_data = preprocess_video(frames)
    predictions = cnn_lstm_model.predict(input_data)
    predicted_class = np.argmax(predictions, axis=1)
    return predicted_class[0]

In [24]:
def bicep_curl_logic():
    def calculate_angle(a, b, c):
        a = np.array(a)
        b = np.array(b)
        c = np.array(c)
        
        ba = a - b
        bc = c - b
        
        mag_ba = math.sqrt(ba[0]**2 + ba[1]**2)
        mag_bc = math.sqrt(bc[0]**2 + bc[1]**2)
        
        radians = math.acos(np.dot(ba, bc) / (mag_bc * mag_ba))
        
        angle = np.abs(radians * 180 / np.pi)
        
        if angle > 180:
            angle = 360 - angle
        
        return angle

    def draw_landmarks(image, landmarks, selected_landmarks, connections):
        for idx in selected_landmarks:
            lm = landmarks[idx]
            h, w, _ = image.shape
            cx, cy = int(lm.x * w), int(lm.y * h)
            cv2.circle(image, (cx, cy), 5, (0, 255, 0), -1)
    
        for connection in connections:
            start_idx, end_idx = connection
            if start_idx in selected_landmarks and end_idx in selected_landmarks:
                start_lm = landmarks[start_idx]
                end_lm = landmarks[end_idx]
    
                start_coords = (int(start_lm.x * w), int(start_lm.y * h))
                end_coords = (int(end_lm.x * w), int(end_lm.y * h))
                cv2.line(image, start_coords, end_coords, (255, 0, 0), 2)
                
    selected_landmarks = [
        mp_pose.PoseLandmark.RIGHT_SHOULDER.value,
        mp_pose.PoseLandmark.RIGHT_HIP.value,
        mp_pose.PoseLandmark.RIGHT_KNEE.value,
        mp_pose.PoseLandmark.RIGHT_ELBOW.value,
        mp_pose.PoseLandmark.RIGHT_WRIST.value,
        mp_pose.PoseLandmark.LEFT_SHOULDER.value,
        mp_pose.PoseLandmark.LEFT_HIP.value,
        mp_pose.PoseLandmark.LEFT_KNEE.value,
        mp_pose.PoseLandmark.LEFT_ELBOW.value,
        mp_pose.PoseLandmark.LEFT_WRIST.value,
    ]
    
    connections = [
        (mp_pose.PoseLandmark.RIGHT_SHOULDER.value, mp_pose.PoseLandmark.RIGHT_HIP.value),
        (mp_pose.PoseLandmark.RIGHT_HIP.value, mp_pose.PoseLandmark.RIGHT_KNEE.value),
        (mp_pose.PoseLandmark.RIGHT_SHOULDER.value, mp_pose.PoseLandmark.RIGHT_ELBOW.value),
        (mp_pose.PoseLandmark.RIGHT_ELBOW.value, mp_pose.PoseLandmark.RIGHT_WRIST.value),
        (mp_pose.PoseLandmark.LEFT_SHOULDER.value, mp_pose.PoseLandmark.LEFT_HIP.value),
        (mp_pose.PoseLandmark.LEFT_HIP.value, mp_pose.PoseLandmark.LEFT_KNEE.value),
        (mp_pose.PoseLandmark.LEFT_SHOULDER.value, mp_pose.PoseLandmark.LEFT_ELBOW.value),
        (mp_pose.PoseLandmark.LEFT_ELBOW.value, mp_pose.PoseLandmark.LEFT_WRIST.value)
    ]
    
    def calculate_progress(angle, min_angle, max_angle):
        progress = 1 - (angle - min_angle) / (max_angle - min_angle)
        progress = max(0, min(progress, 1))
        return progress
    
    cap = cv2.VideoCapture(0)
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        counter = 0
        stage = None
        while cap.isOpened():
            ret, frame = cap.read()
        
            frame = cv2.flip(frame, 1)
        
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            
            results = pose.process(image)
        
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
            try:
                landmarks = results.pose_landmarks.landmark
        
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
        
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
        
                left_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                right_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
                
                torso_angle = calculate_angle(right_shoulder, right_hip, right_knee)
    
                min_angle = 10
                max_angle = 175
                
                cv2.putText(image, f'{left_angle:.1f}', 
                            tuple(np.multiply(right_elbow, [640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        
                progress = calculate_progress(left_angle, min_angle, max_angle)
        
                meter_x_start, meter_y_start = 10, 80
                meter_width, meter_height = 20, 200
                filled_height = int(progress * meter_height)
        
                cv2.rectangle(image, (meter_x_start, meter_y_start),
                              (meter_x_start + meter_width, meter_y_start + meter_height), (200, 200, 200), -1)
                cv2.rectangle(image, (meter_x_start, meter_y_start + meter_height - filled_height),
                              (meter_x_start + meter_width, meter_y_start + meter_height), (0, 255, 0), -1)
    
                cv2.putText(image, f'{int(progress * 100)}%', (meter_x_start + 30, meter_y_start + meter_height),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        
                cv2.putText(image, 'Count: ', (15, 12),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(counter), (10, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
                feedback = []
                if abs(right_elbow[0] - right_shoulder[0]) > 0.1 or abs(left_elbow[0] - left_shoulder[0]) > 0.1:
                    feedback.append("Hands too far apart")
                if left_angle > max_angle:
                    feedback.append("You're going too low")
                if torso_angle < 170:
                    feedback.append("Don't lean forward! Keep your back straight.")
                elif torso_angle > 190:
                    feedback.append("Don't lean backward! Keep your back straight.")
        
                    
                for i, msg in enumerate(feedback):
                    cv2.putText(image, msg, (400, 30 + i * 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
    
                if left_angle > max_angle and right_angle > max_angle:
                    stage = "down"
                if left_angle < min_angle and right_angle < min_angle and stage == "down":
                    stage = "up"
                    counter += 1
                    print("Counter:", counter)
        
            except:
                pass
    
            draw_landmarks(image, landmarks, selected_landmarks, connections)
        
            cv2.imshow('Biceps Curl', image)
        
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()

In [25]:
def squat_logic():
    def calculate_angle_alt(shoulder, hip):
        x1, y1 = shoulder
        x2, y2 = hip
    
        vector = (x1 - x2, y1 - y2)
    
        vertical_vector = (0, -1)
    
        dot_product = vector[0] * vertical_vector[0] + vector[1] * vertical_vector[1]
    
        vector_magnitude = math.sqrt(vector[0]**2 + vector[1]**2)
        vertical_magnitude = 1
    
        angle_rad = math.acos(dot_product / (vector_magnitude * vertical_magnitude))
    
        angle_deg = math.degrees(angle_rad)
    
        return angle_deg
    
    def draw_landmarks(image, landmarks, selected_landmarks, connections):
        for idx in selected_landmarks:
            lm = landmarks[idx]
            h, w, _ = image.shape
            cx, cy = int(lm.x * w), int(lm.y * h)
            cv2.circle(image, (cx, cy), 5, (0, 255, 0), -1)
    
        for connection in connections:
            start_idx, end_idx = connection
            if start_idx in selected_landmarks and end_idx in selected_landmarks:
                start_lm = landmarks[start_idx]
                end_lm = landmarks[end_idx]
    
                start_coords = (int(start_lm.x * w), int(start_lm.y * h))
                end_coords = (int(end_lm.x * w), int(end_lm.y * h))
                cv2.line(image, start_coords, end_coords, (255, 0, 0), 2)
                
    selected_landmarks = [
        mp_pose.PoseLandmark.RIGHT_SHOULDER.value,
        mp_pose.PoseLandmark.RIGHT_HIP.value,
        mp_pose.PoseLandmark.RIGHT_KNEE.value,
        mp_pose.PoseLandmark.RIGHT_ANKLE.value,
        mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value,
        # mp_pose.PoseLandmark.LEFT_SHOULDER.value,
        # mp_pose.PoseLandmark.LEFT_HIP.value,
        # mp_pose.PoseLandmark.LEFT_KNEE.value,
        # mp_pose.PoseLandmark.LEFT_ANKLE.value,
        # mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value,
    ]
    
    connections = [
        (mp_pose.PoseLandmark.RIGHT_SHOULDER.value, mp_pose.PoseLandmark.RIGHT_HIP.value),
        (mp_pose.PoseLandmark.RIGHT_HIP.value, mp_pose.PoseLandmark.RIGHT_KNEE.value),
        (mp_pose.PoseLandmark.RIGHT_KNEE.value, mp_pose.PoseLandmark.RIGHT_ANKLE.value),
        (mp_pose.PoseLandmark.RIGHT_ANKLE.value, mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value),
        # (mp_pose.PoseLandmark.LEFT_SHOULDER.value, mp_pose.PoseLandmark.LEFT_HIP.value),
        # (mp_pose.PoseLandmark.LEFT_HIP.value, mp_pose.PoseLandmark.LEFT_KNEE.value),
        # (mp_pose.PoseLandmark.LEFT_KNEE.value, mp_pose.PoseLandmark.LEFT_ANKLE.value),
        # (mp_pose.PoseLandmark.LEFT_ANKLE.value, mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value)
    ]
    
    cap = cv2.VideoCapture(0)
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        counter = 0
        stage = None
        up = True
        down = False
        while cap.isOpened():
            ret, frame = cap.read()
        
            frame = cv2.flip(frame, 1)
        
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)
        
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
            try:
                landmarks = results.pose_landmarks.landmark
        
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                
                shoulder_hip_vertical = calculate_angle_alt(right_shoulder, right_hip)
                hip_knee_vertical = calculate_angle_alt(right_hip, right_knee)
                knee_ankle_vertical = calculate_angle_alt(right_knee, right_ankle)
                
                cv2.putText(image, f'SH-HIP: {shoulder_hip_vertical:.1f}', 
                            tuple(np.multiply(right_hip, [640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, f'HIP-KNEE: {hip_knee_vertical:.1f}',
                            tuple(np.multiply(right_knee, [640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, f'KNEE-ANKLE: {knee_ankle_vertical:.1f}',
                            tuple(np.multiply(right_ankle, [640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                
                feedback = []
                if shoulder_hip_vertical < 10:
                    feedback.append("Bend forward")
                if shoulder_hip_vertical > 25:
                    feedback.append("Bend backward")
                if 50 < hip_knee_vertical < 80:
                    feedback.append("Lower your hips")
                if hip_knee_vertical > 95:
                    feedback.append("Knees falling over toes")
                if knee_ankle_vertical > 30:
                    feedback.append("Squats too deep")
                
                for i, msg in enumerate(feedback):
                    cv2.putText(image, msg, (10, 100 + i * 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
    
                if up:
                    if hip_knee_vertical < 32:
                        stage = "stage1"  # up
                    elif stage == "stage1" and 35 < hip_knee_vertical < 65:
                        stage = "stage2"  # middle
                    elif stage == "stage2" and 75 < hip_knee_vertical < 95:
                        stage = "stage3"  # down  
                        down = True
                        up = False
                        
                if down:
                    if stage == "stage3" and 35 < hip_knee_vertical < 65:
                        stage = "stage2"
                    elif stage == "stage2" and hip_knee_vertical < 32:
                        stage = "stage1"
                        down = False
                        up = True
                        counter += 1     
    
                cv2.putText(image, 'Count: ', (15, 12),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(counter), (10, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
                    
            except:
                pass
    
            draw_landmarks(image, landmarks, selected_landmarks, connections)
        
            cv2.imshow('Squats', image)
        
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()

In [33]:
import cv2
import mediapipe as mp
import time
import numpy as np

def main():
    # Initialize webcam
    cap = cv2.VideoCapture(1)
    mp_pose = mp.solutions.pose

    # buffers = []
    buffer = []
    buffer_size = 15 
    activity_map = {0: "Bicep Curls", 1: "Squats"}
    current_activity = None
    sequence_count = 0
    # i = 0

    frame_rate = 4  # 5 FPS
    frame_interval = 1 / frame_rate  # Interval between frames in seconds
    last_time = time.time()

    start_time = time.time()
    wait_time = 7  # 15 seconds preparation time

    try:
         while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
                
            frame = cv2.flip(frame, 1)

            elapsed_time = time.time() - start_time
            if elapsed_time < wait_time:
                cv2.putText(frame, "Activity: None", (10, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                cv2.imshow('Activity Recognition', frame)


                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                continue

                
            if elapsed_time < wait_time + 5: 
                cv2.putText(frame, "Start", (10, 90),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.imshow('Activity Recognition', frame)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                continue

            current_time = time.time()
            if current_time - last_time >= frame_interval:
                last_time = current_time  

                buffer.append(frame)
                if len(buffer) > buffer_size:
                    buffer.pop(0)

                if len(buffer) == buffer_size:
                    # sequence_count += 1
                    # buffers.append(buffer.copy())
                    # buffer.clear()
                    # if sequence_count == 2: 
                        sequence_array = np.array(buffer)
                        predicted_class = predict_activity(sequence_array)
                        print(predicted_class)
                        current_activity = activity_map.get(predicted_class, "Detecting...")
                        print(current_activity)

                        if current_activity == "Bicep Curls":
                            bicep_curl_logic()
                            break
                        elif current_activity == "Squats":
                            squat_logic()
                            break
                            
                cv2.putText(frame, f"Activity: {current_activity or 'Collecting...'}", (10, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            cv2.imshow('Activity Recognition', frame)
 
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                 
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 627ms/step
1
Squats
An error occurred: cannot access local variable 'landmarks' where it is not associated with a value


In [31]:
import cv2
import mediapipe as mp
import math

# Initialize Mediapipe Pose module
mp_pose = mp.solutions.pose

# Function to calculate the angle between three points (using the law of cosines)
def calculate_angle(a, b, c):
    # a, b, c are (x, y) coordinates of three points
    # Vector AB and BC
    ab = [b[0] - a[0], b[1] - a[1]]
    bc = [b[0] - c[0], b[1] - c[1]]
    
    # Dot product and magnitudes
    dot_product = ab[0] * bc[0] + ab[1] * bc[1]
    magnitude_ab = math.sqrt(ab[0] ** 2 + ab[1] ** 2)
    magnitude_bc = math.sqrt(bc[0] ** 2 + bc[1] ** 2)
    
    # Calculate angle in radians
    angle_rad = math.acos(dot_product / (magnitude_ab * magnitude_bc))
    # Convert to degrees
    angle_deg = math.degrees(angle_rad)
    return angle_deg

# Function to classify the activity based on angles
def classify_activity(frame, landmarks):
    # Get the coordinates of the relevant body parts
    shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y]
    elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].y]
    wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].y]
    
    hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y]
    knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].y]
    ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].y]
    
    # Check if the knee landmark exists
    knee_detected = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].visibility > 0.5
    
    if knee_detected:
        # Squat Logic (if knee is detected)
        knee_angle = calculate_angle(hip, knee, ankle)
        activity = "Squatting"
        cv2.putText(frame, f'Knee Angle: {knee_angle:.1f}', 
                    (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    else:
        # Bicep Curl Logic (if knee is NOT detected)
        elbow_angle = calculate_angle(shoulder, elbow, wrist)
        activity = "Bicep Curling"
        cv2.putText(frame, f'Elbow Angle: {elbow_angle:.1f}', 
                    (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    return activity

# Main program
def main():
    cap = cv2.VideoCapture(0)
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Flip the frame for better visualization
            frame = cv2.flip(frame, 1)

            # Process the frame to detect landmarks
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image_rgb)

            # If landmarks are found
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark

                # Classify activity based on landmarks
                activity = classify_activity(frame, landmarks)
                
                if activity == "Bicep Curling":
                    bicep_curl_logic(frame, pose, cap)
                elif activity == 'Squatting':
                    squat_logic(frame, pose, cap)

                # Display the activity on the frame
                cv2.putText(frame, f"Activity: {activity}", (10, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            # Show the frame with the activity label
            cv2.imshow('Activity Recognition', frame)

            # Exit on pressing 'q'
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


TypeError: squat_logic() takes 0 positional arguments but 3 were given

In [6]:
import time
start_time = time.time()
wait = 5

while(1):
    current_time = time.time()
    if  int(current_time) - int(start_time)  == wait:
        break
        
print(int(current_time)- int(start_time))

5


In [1]:
print(time.time() )

NameError: name 'time' is not defined

In [29]:
import cv2

index = 0
while True:
    cap = cv2.VideoCapture(index)
    if not cap.isOpened():
        break
    print(f"Camera index {index} is available.")
    index += 1
    cap.release()


Camera index 0 is available.
Camera index 1 is available.
